# Import lib

In [ ]:
!pip install pythainlp -qq
!pip install epitran -qq
!pip install sklearn_crfsuite -qq
!pip install tensorflow deepcut -qq
!pip install attacut -qq
!pip install deepcut -qq

     |████████████████████████████████| 11.5 MB 4.9 MB/s 
     |████████████████████████████████| 148 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 37.7 MB/s 
     |████████████████████████████████| 73 kB 1.2 MB/s 
     |████████████████████████████████| 743 kB 5.1 MB/s 
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 462 kB 56.4 MB/s 
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 473 kB 39.3 MB/s 
     |████████████████████████████████| 87 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 45 kB 1.4 MB/s 


In [ ]:
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
import re
import string
import os
from tqdm import tqdm
import glob
import pythainlp
from pythainlp import sent_tokenize, word_tokenize
import numpy as np
import pandas as pd

#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

# **Common function**

In [ ]:
#Edit disntance from geek
def editDistance(str1, str2, m, n):

	# If first string is empty, the only option is to
	# insert all characters of second string into first
	if m == 0:
		return n

	# If second string is empty, the only option is to
	# remove all characters of first string
	if n == 0:
		return m

	# If last characters of two strings are same, nothing
	# much to do. Ignore last characters and get count for
	# remaining strings.
	if str1[m-1] == str2[n-1]:
		return editDistance(str1, str2, m-1, n-1)

	# If last characters are not same, consider all three
	# operations on last character of first string, recursively
	# compute minimum cost for all three operations and take
	# minimum of three values.
	return 1 + min(editDistance(str1, str2, m, n-1), # Insert
				editDistance(str1, str2, m-1, n), # Remove
				editDistance(str1, str2, m-1, n-1) # Replace
				)

# Driver code
str1 = "เปลี่ยน"
str2 = "เปลี่ย"
print (editDistance(str1, str2, len(str1), len(str2)))

# This code is contributed by Bhavya Jain


1


# Corpus

In [ ]:
from pythainlp.spell import NorvigSpellChecker
from pythainlp.corpus import download , get_corpus_path , get_corpus

In [ ]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
len(checker.dictionary())
checker.dictionary()

dict_items([('เว็จ', 2), ('กอ.รมน.', 74), ('ไอยรา', 315), ('ร้องไห้', 2774), ('ชนม', 17), ('ตป', 8), ('อับราฮัม', 28), ('ศาลเพียงตา', 7), ('สปน', 2), ('ยันต์', 224), ('อัลเบอร์', 5), ('เอ้ด', 2), ('ถู', 703), ('ทำครัว', 86), ('รอล', 209), ('ถอนตัว', 384), ('อตีต', 2), ('ปาจารย์', 2), ('ของหวาน', 301), ('มอมแมม', 133), ('มรณภาพ', 112), ('โรยรา', 45), ('ทหารไทย', 127), ('กฤษฎ์', 11), ('เหยอ', 6), ('ประพิณ', 3), ('พึ่งพิง', 243), ('มากมาย', 8757), ('เตร๊าส์', 5), ('เบิร์ธเดย์', 30), ('พิมพาภรณ์', 11), ('ปึ้ก', 44), ('มอร์', 201), ('ลูกไม้', 219), ('เอนทิตี', 89), ('ฟืด', 28), ('วิระ', 7), ('ราล์ฟ', 46), ('วิม', 82), ('นาซี', 155), ('อุปัฏฐาก', 29), ('ดิ่ง', 760), ('ทแยง', 46), ('หนีเสือ', 23), ('ประดาป', 6), ('คร้าง', 2), ('ฝาละมี', 9), ('อ๊าาาา', 17), ('สนธิ์', 13), ('เสี่ยง', 3834), ('เซลส์', 73), ('กะบังลม', 15), ('ทริปป์', 4), ('วิทยาการ', 74), ('ไลซ์', 11), ('ธาตุเบา', 5), ('เด็ม', 6), ('อัดสำเนา', 26), ('เดวิส', 68), ('คเตท', 5), ('โฮ่', 157), ('สเส็ท', 2), ('เพ็ก', 24), ('ตรึง', 55

# Dictionary

In [ ]:
from pythainlp.corpus import download , get_corpus_path , get_corpus

In [ ]:
class unigram():

  def __init__(self,dict_path):    
    self.word = [item[0] for item in dict_path]
    self.freq = [item[1] for item in dict_path]
    self.df = pd.DataFrame(self.word,columns=["words"])
    self.df["freq"] = pd.DataFrame(self.freq)
    self.dictt = self.makedict()

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,word):
    return self.df.loc[word]
  
  def makedict(self):
    tmp_dict ={}
    for i in range(len(self.word)):
      tmp_dict[self.word[i]] = self.freq[i]

    return tmp_dict

  

In [ ]:
class corpus():
  def dow(self,name):
    download(name)
    path = get_corpus_path(name)
    cor = get_corpus(path)
    self.word = ["".join(x.split("\t")[:-1]) for x in cor]
    self.fre = ["".join(x.split("\t")[-1]) for x in cor]
    dict_cor = {"word":self.word,"fre":self.fre}
    self.df = pd.DataFrame(dict_cor,index=range(len(self.word))).set_index('word')
    self.dictt = self.to_dict()
  
  def __init__(self,name):
    self.dow(name)

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,word):
    return self.df.loc[word]
  
  def to_dict(self):
    tmp_dict={}
    for i in tqdm(range(len(self.word))):
      tmp_dict[self.word[i]] = self.fre[i]
    return tmp_dict

In [ ]:
class candidater():
  def edits1(self,word):
    "All edits that are one edit away from `word`."    
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    # transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    #replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    # inserts    = [L + c + R               for L, R in splits for c in letters]   
    return set(deletes)#set(deletes + transposes + replaces + inserts)


  # def edits2(self,word):
  #   "All edits that are two edit away from `word`."    
  #   splits     = [(word[:i], word[i:])    for i in range(len(word) + 2)]
  #   deletes    = [L + R[2:]               for L, R in splits if R]
  #   # transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
  #   #replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
  #   # inserts    = [L + c + R               for L, R in splits for c in letters]   
  #   return set(deletes)#set(deletes + transposes + replaces + inserts)

  def delet2dict(self,dict_new,text):   
    for delet1 in self.edits1(text):
      if dict_new.get(delet1) is None:
        dict_new.update({
            delet1:{text}
            })
      else:
        dict_new.get(delet1).add(text)

  def __init__(self,unigrame):
    self.unidict = unigrame.dictt
    uniword = unigrame.word    
    dict_new = {}
    for text in tqdm(uniword):     
      self.delet2dict(dict_new,text)
    self.dict_new = dict_new

  def candidate(self,word):
    set_candidate = set()

    if word in self.dict_new.keys():
      set_candidate.update(self.dict_new[word])
      
    del_set = self.edits1(word)

    for word_del in del_set:
      
      if word_del in self.unidict.keys():
        set_candidate.add(word_del)

      if word_del in self.dict_new.keys():
        set_candidate.update(self.dict_new[word_del])
    return set_candidate

In [ ]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
unigrame = unigram(checker.dictionary())

In [ ]:
trigrame = corpus("tnc_trigram_word_freqs")
bigrame = corpus("tnc_bigram_word_freqs")

Corpus: tnc_trigram_word_freqs
- Downloading: tnc_trigram_word_freqs 2017


100%|██████████| 3704931/3704931 [00:03<00:00, 1094289.12it/s]


Corpus: tnc_bigram_word_freqs
- Downloading: tnc_bigram_word_freqs 2017


100%|██████████| 1326915/1326915 [00:01<00:00, 1177979.38it/s]


In [ ]:
candit = candidater(unigrame)

100%|██████████| 39964/39964 [00:01<00:00, 23326.65it/s]


In [ ]:
%timeit candit.candidate('สงัดดี')

The slowest run took 5.87 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 6.69 µs per loop


In [ ]:
candit.candidate('สงัดดี')

set()

# **Combine function to use for full senetnce**

In [ ]:
class Correction:
  def __init__(self,sentences):
    self.input = sentences    

  def find_trigram_words(self):
    """ find all init set of trigrames"""
    word_deepcut =  word_tokenize(self.input, engine="deepcut")

  def find_index_setof_


# Function area

In [ ]:
#combine function for find correct words
def correct_word_s(input_words):  
  #2. find possible word for each word
  word__wrong_list = input_words

  record_possible_word =[]
  
  
  for word in tqdm(word__wrong_list):
    # each_word_list =[]
    each_word_list = candit.candidate(word)
    record_possible_word.append(each_word_list)  

  #3.combine each possible words to trigram word

  if len(record_possible_word) != 0:
    all_edit_words =[]
    for i in tqdm(range(len(word__wrong_list))):
      # print(i)  
      for item in record_possible_word[i]:
        if i == 0:
          new_word = item + word__wrong_list[1] + word__wrong_list[2]
        elif i == 1:
          new_word = word__wrong_list[0] + item + word__wrong_list[2]
        else:
          new_word = word__wrong_list[0] + word__wrong_list[1] + item
        all_edit_words.append(new_word)
        
  else:
    print('no record possible word')

  #4. loop edit word in dict again to find most freq(prob)

  selected_words=[]
  freq_words=[]

  for item in tqdm(all_edit_words):
    try:
      freq_words = trigrame.dictt[item]
      selected_words.append(item)
      freq_words.append(int(freq_words))
    except:
      pass   

  if len(selected_words) == 0:
    for item in tqdm(all_edit_words):  
      try:
        freq_words = bigrame.dictt[item]
        selected_words.append(item)
        freq_words.append(int(freq_words))
      except:
        pass

  #after augment may be reduce to 1 words (unigrame dict)
  if len(selected_words) == 0:
    for item in tqdm(all_edit_words):  
      if item in unigrame.word:
        selected_words.append(item)
        freq_words.append(unigrame[unigrame.df['words'] == item].freq)

  #select the most freq
  try:
    correct_word = selected_words[np.argmax(freq_words)]
    return correct_word
  except: #may be correct but wrong cut that why we cannot find it in dict   
    return "".join(input_words)

**Full sentence run**

In [ ]:
#combine all
%%time
text = "แปลแต่จริงสิ่งนี้ไม่น่าจะเกิดขึ้นกันฉันบอกกับฉันว่าเธอทำได้ยังไงเปลี่ยนคนที่คอปิดหัวใจให้รักเทอง่ายตาย"
word_deepcut =  word_tokenize(text, engine="deepcut") #deep or atta

full_word=[]
Wrong_word=[]

#index
start_idx_list=[]
end_idx_list=[]

#step1. 
count = 0

for idx in range(len(word_deepcut)):
  if idx == 0:
    word=word_deepcut[idx:idx+2]

    start_idx_list.append(count)
    count_word=''.join(word)
    count+=len(count_word)
    end_idx_list.append(count)
    
    count_word=''.join(word)
    word.insert(0,'<s/>')
    full_word.append(word)

  if idx == len(word_deepcut)-2:
    word=word_deepcut[idx:idx+2]
    count_word=''.join(word)
    word.append('<s/>')

    count+=1
    start_idx_list.append(count)
    count+=len(count_word)
    end_idx_list.append(count)
    full_word.append(word)
    break
  else:
    word=word_deepcut[idx:idx+3]
    count_word=''.join(word)

    count+=1
    start_idx_list.append(count)
    count+=len(count_word)
    end_idx_list.append(count)

  full_word.append(word)

# print(full_word)

#step2.
wrong_words =[]
correct_words =[]
wrong_word_combine=[]
selected_start_index=[]
selected_end_index=[]

for i in range(len(full_word)):
  tri_word="".join(full_word[i])
  if tri_word in trigrame.word:
    # correct_words.append(item)
    pass   
  else:    
    wrong_words.append(full_word[i])
    wrong_word_combine.append(tri_word)
    selected_start_index.append(start_idx_list[i])
    selected_end_index.append(end_idx_list[i])

print(wrong_words)
edit_word_list=[]

#find correct word
for item in wrong_words:
  correct_one = correct_word_s(item)
  edit_word_list.append(correct_one)
  print(correct_one)

#find dict list
result =[]
for i in range(len(edit_word_list)):
  edit_word ={}
  edit_word["start"] = selected_start_index[i]
  edit_word["end"] = selected_end_index[i]
  edit_word["old_word"] = wrong_word_combine[i]
  edit_word["new_word"] = edit_word_list[i]
  result.append(edit_word)

[['<s/>', 'ตา'], ['ตา']]


 50%|█████     | 1/2 [00:00<00:00, 8701.88it/s]


IndexError: ignored

In [ ]:
result

[{'end': 6, 'new_word': '<s/>แลแต่', 'old_word': '<s/>แปลแต่', 'start': 0},
 {'end': 17, 'new_word': 'แปลกแต่จริง', 'old_word': 'แปลแต่จริง', 'start': 7},
 {'end': 29,
  'new_word': 'แต่จริงสิ่ง',
  'old_word': 'แต่จริงสิ่ง',
  'start': 18},
 {'end': 41,
  'new_word': 'จริงสิ่งนี้',
  'old_word': 'จริงสิ่งนี้',
  'start': 30},
 {'end': 115,
  'new_word': 'ขึ้นกับฉัน',
  'old_word': 'ขึ้นกันฉัน',
  'start': 105},
 {'end': 125, 'new_word': 'กันฉันบอก', 'old_word': 'กันฉันบอก', 'start': 116},
 {'end': 201, 'new_word': 'ได้ยังไว', 'old_word': 'ได้ยังไง', 'start': 193},
 {'end': 214,
  'new_word': 'ยังไงเปลี่ยน',
  'old_word': 'ยังไงเปลี่ยน',
  'start': 202},
 {'end': 226,
  'new_word': 'ไงเปลี่ยนคน',
  'old_word': 'ไงเปลี่ยนคน',
  'start': 215},
 {'end': 247, 'new_word': 'ขนที่คอ', 'old_word': 'คนที่คอ', 'start': 240},
 {'end': 256, 'new_word': 'ที่รอปิด', 'old_word': 'ที่คอปิด', 'start': 248},
 {'end': 267,
  'new_word': 'คอปิดหัวใจ',
  'old_word': 'คอปิดหัวใจ',
  'start': 257},
 {'end': 